<a href="https://colab.research.google.com/github/sivanv-unbxd/colab-templates/blob/A2C_import_template/Pepper_X_A2C_Import_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## This notebook contains all the information and utilities required for performing an Import of catalog or other Product information into PIM.

### Pepper-X SDK utilities

In [ ]:
from django.core.exceptions import ObjectDoesNotExist
import logging
from django.conf import settings
import requests
import json
from common.utils.utility import get_pim_app_domain, get_pas_domain, get_pepperx_domain
logger = logging.getLogger(__name__)

class App(object):
    def __init__(self, app_id="", app_name=""):
        self.app_id= app_id
        self.app_name = app_name
        self.get(app_id, app_name)

    def create(self, app_id, name, credentials={}):
        try:
            url = f"{get_pepperx_domain()}api/v1/app_data/"
            payload = json.dumps({
                "app_id": app_id,
                "app_name": name,
                "app_creds" : credentials
            })
            headers = {
                'Content-Type': 'application/json'
            }
            response = requests.request("POST", url, headers=headers, data=payload)
            data =response.text
            if response.status == 200:
                return data
            else:
                return None

        except ObjectDoesNotExist:
            pass

    def get(self, app_id, app_name=""):
            try:
                url = f"{get_pepperx_domain()}api/v1/app_data/"
                payload = json.dumps({
                    "app_id": app_id,
                })
                headers = {
                    'Content-Type': 'application/json'
                }
                response = requests.request("GET", url, headers=headers, data=payload)
                data =response.text
                if response.status_code == 200:
                    app_data =  json.loads(data)
                    app_data = app_data["data"]
                    self.app_creds = app_data["app_data"]["app_creds"]
                    self.app_name = app_data["app_data"]["app_name"]
                    self.app_id = app_data["app_data"]["app_id"]
                    return app_data
                else:
                    return None

            except ObjectDoesNotExist:
                pass


class AppUser(object):
    def __init__(self, app_id, identifier):
        self.app_id= app_id
        self.identifier= identifier



    def create(self, credentials={}, pim_creds ={}):
        try:
            url = f"{get_pepperx_domain()}api/v1/app_user_data/"
            payload = json.dumps({
                "app_id": self.app_id,
                "identifier": self.identifier,
                "user_creds" : credentials,
                "pim_creds": pim_creds
            })
            headers = {
                'Content-Type': 'application/json'
            }
            response = requests.request("POST", url, headers=headers, data=payload)
            data =response.text
            if response.status_code in [200, 201]:
                app_data =  json.loads(data)
                app_data = app_data["data"]
                self.app_user_creds = app_data["app_user"]["app_creds"]
                self.pim_creds = app_data["app_user"].get("pim_creds", None)
            else:
                return data


        except ObjectDoesNotExist:
            pass

    def get(self):
        try:
            url = f"{get_pepperx_domain()}api/v1/app_user_data/"
            payload = json.dumps({
                "app_id": self.app_id,
                "identifier" : self.identifier
            })
            headers = {
                'Content-Type': 'application/json'
            }
            response = requests.request("GET", url, headers=headers, data=payload)
            data = response.text
            if response.status_code == 200:
                app_data = json.loads(data)
                app_data = app_data["data"]
                return app_data
            else:
                return None
        except ObjectDoesNotExist:
            pass

class AppUserPIM(object):
    def __init__(self, api_key=""):

        self.api_key = api_key

    def get(self):
        try:
            url = f"{get_pepperx_domain()}api/v1/app_user_pim_data/"

            payload = json.dumps({
                "api_key": self.api_key
            })
            headers = {
                'Content-Type': 'application/json'
            }
            response = requests.request("GET", url, headers=headers, data=payload)
            data =response.text
            if response.status_code == 200:
                app_data = json.loads(data)
                app_data = app_data["data"]
                # self.app_creds = app_data["credentials"]
                # self.app_name = app_data["app_data"]["name"]
                # self.app_id = app_data["app_data"]["label"]
                return app_data
            else:
                return None

        except ObjectDoesNotExist:
            pass

### Import n-D to 1-D Transformer

In [ ]:
import json
import logging
from datetime import datetime, timezone


logger = logging.getLogger('shopify_catalog.product.transformer_graphql')


class Transformer:
    logger = logging.getLogger('shopify_catalog/product/app_to_pim/denormalisation.json')

    def __init__(self, configpath):
        self.logger.info('Loading transformer configuration from: %s', configpath)
        with open(configpath, 'r') as fp:
            self.config = json.load(fp)

    def _update_product(product, key, data):
        if type(data) is dict:
            product.update(data)
        else:
            product[key] = data



    def transform(self, product,category_full_path_map, product_variants = ""):
        changed_product = {}

        for fieldname, fieldvals in product.items():
            print(fieldname)
            if fieldname in self.config:
                transformer = self.config[fieldname]
                helper_func = getattr(self, transformer['helper'])
                helper_params = {}
                if "helper_params" in transformer:
                    helper_params = transformer['helper_params']

                self._update_product(changed_product, transformer['key'],
                                     helper_func(product, fieldname, helper_params))

            elif fieldname == "combination":
                helper_func = getattr(self, "handle_variant")
                helper_params = product_variants
                self._update_product(changed_product, fieldname,
                                     helper_func(product, fieldname, helper_params))

            elif fieldname == "categories_ids":
                helper_func = getattr(self, "handler_a2c_ids")
                helper_params = category_full_path_map
                self._update_product(changed_product, fieldname,
                                     helper_func(product, fieldname, helper_params))

            elif fieldvals != None:
                print(type(fieldvals))
                if isinstance(fieldvals, list):
                    helper_func = getattr(self, "handler_a2c_array_dict")
                    helper_params = {"prefix": fieldname}
                    self._update_product(changed_product, fieldname,
                                         helper_func(product, fieldname, helper_params))

                elif isinstance(fieldvals, dict):
                    helper_func = getattr(self, "handle_dict")
                    helper_params = {"prefix" : fieldname}
                    self._update_product(changed_product,fieldname,
                                         helper_func(product, fieldname, helper_params))


                else:
                    changed_product[fieldname] = fieldvals

        if self.check_dimensionality(changed_product):
            completely_changed_product = {prod_key: val for prod_key, val in changed_product.items() if val not in [None, ""]}
            handle_bool_props = ["is_free_shipping", "is_condition_shown", "in_stock", "manage_stock", "availability"]
            final_dict = completely_changed_product
            for key, value in completely_changed_product.items():

                if key in handle_bool_props:
                    final_dict = handle_bool(key, completely_changed_product)
            return final_dict

        else:
            return "Improperly transformed"

    def check_dimensionality(self, changed_product):
        values = list(changed_product.values())
        # if isinstance(values, dict):
        #     return False
        # elif isinstance(values, list):
        #     for values_inside_list in values:
        #         if isinstance(values_inside_list, dict) or isinstance(values_inside_list, list):
        #             return False
        #     return True
        for values_inside_list in values:
            if isinstance(values_inside_list, dict):
                return False
            elif isinstance(values_inside_list,list):
                for values_inside_value in values_inside_list:
                    if isinstance(values_inside_value, dict) or isinstance(values_inside_value, list):
                             return False
                return True
            else:
                return True

    def handle_tier_price(product, key, params):
        pass
    def handle_variant(product, key, params):
        # prefix = params["prefix"] if "prefix" in params else ""
        product_variants = params
        option_id_hash = {}
        option_items_id_hash = {}
        for variable in product_variants:
            option_id_hash[variable["product_option_id"]] = {"name": variable["name"], "type": variable["type"]}
            for option_items in variable["option_items"]:
                option_items_copy = option_items.copy()
                option_items_copy.pop("id")
                full_desc_value = option_items_copy.pop("additional_fields")
                for full_desc_key, value in full_desc_value.items():
                    option_items_copy[key] = value
                option_items_id_hash[option_items["id"]] = option_items_copy

        changed_product = {}
        for variant_properties in product[key]:
            #changed_product[option_id_hash[variant_properties['option_id']]['name']] = option_items_id_hash[variant_properties['option_value_id']]['name']
            changed_product['variant_option_type'] = option_id_hash[variant_properties['option_id']]['type']
            for property_key, property_value in option_items_id_hash[variant_properties["option_value_id"]].items():
                changed_product['variant_option_' + option_id_hash[variant_properties['option_id']]['name'] + '_' + property_key] = property_value

            changed_product['variant_option_' + option_id_hash[variant_properties['option_id']]['name']+ '_' + 'name'] = changed_product.pop('variant_option_' + option_id_hash[variant_properties['option_id']]['name'] + '_' + 'name')
            # changed_product['variant_option_' + option_id_hash[variant_properties['option_id']]['name']] = changed_product.pop('variant_option_' + option_id_hash[variant_properties['option_id']]['name'] + '_' + 'name')


        return changed_product


    def handle_dict(product, key, params):
        # TODO handle methods
        prefix = str(params["prefix"] + '_') if "prefix" in params else ""
        changed_product = {}
        if product[key] != None:
            for key, value in product[key].items():
                if value != None:
                    changed_product[prefix + str(key)] = value
            return changed_product


    def handler_datetime(product, key, params):
        if product[key] is not None:
            try:
                dt = datetime.fromisoformat(product[key])
            except ValueError:
                dt = datetime.utcnow()
            dt = dt.astimezone(tz=timezone.utc)
            return dt.strftime('%Y-%m-%dT%H:%M:%SZ')
        else:
            return None

    def handler_string(product, key, params):
        return str(product[key]).strip()

    def handler_decimal(product, key, params):
        try:
            value = float(product[key])
        except (ValueError, TypeError):
            value = None
        return value

    def handler_a2c_date(product, key, params):

        return product[key]["value"] if "value" in product[key] else None

    def handler_a2c_array_dict(product, key, params):
        if len(product[key]):
            if not isinstance(product[key][0], dict):
                return product[key]
            else:
                counter = 1
                changed_product = {}
                if "accepted_keys" not in list(params.keys()):
                    params["accepted_keys"] = []
                for sub_properties in product[key]:
                    sub_property_key_list = list(sub_properties.keys())
                    for sub_property_key in sub_property_key_list:
                        if sub_property_key in params["accepted_keys"]:
                            prefix = str(params["prefix"] + "_") if "prefix" in params else ""
                            if sub_properties[sub_property_key] != None:
                                changed_product[prefix + str(counter) + '_' + str(sub_property_key)] = sub_properties[sub_property_key]
                    counter += 1
                return changed_product

    def handler_a2c_ids(product, key, params):
        if key == "categories_ids":
            changed_product = {}
            changed_product["category"] = []

            for category_code in product[key]:
                changed_product["category"].append(params[category_code])

            return changed_product
        else:
            pass

def handle_bool(key, product):

    if isinstance(product[key], str):
        if product[key] in ["False",'0','false', 0, 'unavailable']:
            product[key] = False
        else:
            product[key] = True
    return product

def buildCategoryTree(self, catData):
    categoryFullPath = dict()

    for cat in catData:
        try:
            if (cat["parent_id"] == 0):
                categoryFullPath[cat["id"]] = cat["name"]
            elif (categoryFullPath[cat["parent_id"]]):
                categoryFullPath[cat["id"]] = categoryFullPath[cat["parent_id"]] + ">" + cat["name"]
        except Exception as e:
            print(e)

    return categoryFullPath

### File Operation Utilities


In [ ]:
import os
import csv
import json
import zipfile
import tempfile
from time import time
import logging
import boto3
from botocore.exceptions import ClientError


def create_tmp_file(data):
    filename = tempfile.NamedTemporaryFile(delete=False).name
    with open(filename, "wb") as f:
        f.write(data)
    return filename


def write_csv_file(data, delimiter="\t"):
    """
        Method write array data to the file and return the file name
    :param data: (list)- Array of array data i.e is supposed to be written to the file.
    :param delimiter:
    :return:
    """
    filename = tempfile.NamedTemporaryFile(delete=False).name
    with open(filename, 'w') as csvfile:
        csvwriter = csv.writer(csvfile, delimiter=delimiter)
        csvwriter.writerows(data)
    return filename


def read_csv_file(data_file, delimiter=","):
    with open(data_file, 'r') as text:
        csv_reader = csv.reader(text, delimiter=delimiter)
        for row in csv_reader:
            yield row


def read_csv_stream(stream, delimiter="\t"):
    csv_reader = csv.reader(iter(stream), delimiter=delimiter)
    for row in csv_reader:
        yield row


def compress_file_to_zip(zip_file_name, local_file_path, arcname):
    """
        Function to compress a file to a zip file.
    :param zip_file_name: (string) - Zipped file name
    :param local_file_path: (string) - File's path on the system.
    :param arcname: (string)  File name without the directory names
    :return: None
    """

    dir_name = os.path.dirname(zip_file_name)
    # check if the directory is not there
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)
    print("### In zipping file from {}  to {}",local_file_path, zip_file_name)
    with zipfile.ZipFile(
        zip_file_name, 'w', zipfile.ZIP_DEFLATED
    ) as zipped_items:
        zipped_items.write(local_file_path, arcname=arcname)


def write_to_file(data, file_path, mode="w", is_json=True):
    """
        Function to write json data fil
    :param data:
    :param file_path: (string) - absolute file path
    :return:
    """
    dir_name = os.path.dirname(file_path)
    # check if the directory is not there
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)
    print("File path in os == > ", file_path)
    try:
        with open(file_path, mode) as outfile:
            if is_json:
                json.dump(data, outfile)
            else:
                outfile.write(data)
    except Exception as e:
        raise Exception("Exception {}".format(str(e)))


def write_feed_file(file_dir, products, app_name=""):
    current_time = round(time())
    file_name = '{app_name}_to_PIM_feed_{timestamp}.json'.format(
        timestamp=current_time, app_name = app_name
    )
    zip_file_name = '{app_name}_to_PIM_feed_{timestamp}.zip'.format(
        timestamp=current_time, app_name = app_name
    )
    file_path = os.path.join(file_dir, file_name)
    with open(file_path, 'w') as f:
        f.write(json.dumps(products))
    zip_file_path = os.path.join(file_dir, zip_file_name)
    compress_file_to_zip(zip_file_path, file_path, file_name)
    return file_name

def extract_zip_file(zip_file_with_path):
    # opening the zip file in READ mode
    with zipfile.ZipFile(zip_file_with_path, 'r') as zip:
        # printing all the contents of the zip file
        files_in_zip = []
        for files in zip.filelist:
            if '.' in files.filename:
                files_in_zip.append(files.filename)
        zip.printdir()
        # extracting all the files
        print('Extracting all the files now...')
        zip.extractall()
        return files_in_zip


def handle_dict(dict_object, params):
    # TODO handle methods
    # Params: [accepted_keys, ignore_keys, required_key,prefix, suffix, op_type]
    op_dict = {}
    item_key = ""
    for key, value in dict_object.items():
        if "accepted_keys" in params and key in params["accepted_keys"]:
            # if all(item_key in sub for sub in [dict_object[key], params["accepted_keys"]]):
                op_dict[key] = dict_object[key]
        else:
            # TODO
            print("Handle all param types")
            #op_dict = dict_object

    return op_dict


def upload_to_s3(file_path, filename, bucket, object_name=None):

        """Upload a file to an S3 bucket

        :param file_name: File to upload
        :param bucket: Bucket to upload to
        :param object_name: S3 object name. If not specified then file_name is used
        :return: True if file was uploaded, else False
        """
        region = os.environ['aws_region']
        key = "app-uploads/" + filename
        # If S3 object_name was not specified, use file_name
        if object_name is None:
            object_name = filename
        s3 = boto3.resource(
            service_name='s3',
            region_name=region,
            aws_access_key_id=os.environ['aws_access_key_id'],
            aws_secret_access_key=os.environ['aws_secret_access_key']
        )

        try:
            s3.Bucket(bucket).upload_file(Filename=file_path+filename, Key=key)
        # Upload the file
        # s3_client = boto3.client('s3')
        # try:
        #     response = s3_client.upload_file(filename_with_path, bucket, object_name)
            url = f"https://{bucket}.s3.{region}.amazonaws.com/{key}"
        except ClientError as e:
            logging.error(e)
            return False
        return url

### Category Module

In [ ]:
from __future__ import print_function
import time
import swagger_client
from swagger_client.rest import ApiException
from pprint import pprint


class BuildCategoryTree:
    def __init__(self, catList, a2c_api_key, a2c_store_key):
        self.catList = catList
        self.categoryTreePath = dict()
        self.a2c_api_key = a2c_api_key
        self.a2c_store_key = a2c_store_key

    def build_categories(self):

        dictfilt = lambda x, y: dict([(i, x[i]) for i in x if i in set(y)])
        categoriesList = []
        cat_keys = ["id", "name", "parent_id"]
        for cat in self.catList:
            categoriesList.append(dictfilt(cat, cat_keys))

        self.categoryTreePath = self.buildCategoryTree(categoriesList)
        return self.categoryTreePath

    #TODO [
    # {"id": 1, "name": pant,"parent_id":0},
    # {"id": 2, "name": shirt,"parent_id":1},
    # {"id": 3, "name": shorts,"parent_id":2}
    # ]

    def buildCategoryTree(self, catData):
        categoryFullPath = self.categoryTreePath
        for cat in catData:
            if (cat["parent_id"] == '0'):
                categoryFullPath[cat["id"]] = cat["name"]
        for cat in catData:
            try:
                if (cat["parent_id"] == '0'):
                    categoryFullPath[cat["id"]] = cat["name"]
                elif (categoryFullPath[cat["parent_id"]]):
                    categoryFullPath[cat["id"]] = categoryFullPath[cat["parent_id"]] + ">" + cat["name"]
            except Exception as e:
                print(e)

        return categoryFullPath
    #TODO categoryFullPath = {1:pant, 2: pant>shirt, 3: pant>shirt>shorts}
    #

    #TODO Name to ID
    def getCategoryIds(self, cats):
        #     print("#### Categories", cats)
        catArr = []
        if (cats != None):
            for cat in cats:
                catId = self.getCategoryId(cat)
                catArr.append(catId)
        print("$$$$$ updated categories", catArr)

        return catArr


    def getCategoryId(self, catName):
        try:
            catId = list(self.categoryTreePath.keys())[list(self.categoryTreePath.values()).index(catName)]
        except Exception as e:
            #         print(e)
            print("Need to create", catName)
            cat_path = catName.split(">")
            parent_id = 0
            name = ""
            for cat_node in cat_path:
                name += cat_node
                print("!!!!!inside for loop cat_node   ", cat_node, " ----- full cat path is ==> ", name)
                try:
                    print("checking cat id for ", name)
                    parent_id = list(self.categoryTreePath.keys())[list(self.categoryTreePath.values()).index(name)]
                except  Exception as e:
                    print("creating category with name as ==> ", cat_node, " and parent id as ==>", parent_id)
                    #TODO Create your A2C cat create here
                    # Configure API key authorization: api_key
                    configuration = swagger_client.Configuration()
                    configuration.api_key['api_key'] = self.a2c_api_key
                    configuration.api_key['store_key'] = self.a2c_store_key
                    # Uncomment below to setup prefix (e.g. Bearer) for API key, if needed
                    # configuration.api_key_prefix['store_key'] = 'Bearer'

                    # create an instance of the API class
                    category_create_instance = swagger_client.CategoryApi(swagger_client.ApiClient(configuration))
                    name = cat_node  # str | Defines category's name that has to be added
                    parent_id = parent_id  # str | Adds categories specified by parent id (optional)
                    category_create_response = category_create_instance.category_add(name, parent_id=parent_id)
                    pprint(category_create_response)


                    print("****** created category is ==>", str(name))
                    dictionary = {"id":category_create_response.result.category_id, "name": name, "parent_id": parent_id}
                    parent_id = category_create_response.result.category_id
                    # [{"id": 1, "name": pant,"parent_id":0}]

                    self.categoryTreePath = self.buildCategoryTree([dictionary])

                    catId = parent_id
                name += ">"
        return catId

In [ ]:
from __future__ import print_function
import time
import swagger_client
from swagger_client.rest import ApiException
from pprint import pprint
import requests
from common.PAS.app import AppUserPIM
import common.utils.file_operations
from .denormalisation import Transformer
from traceback import print_exc
from common.utils.file_operations import write_feed_file
import json


class ProductImporter(object):

    def __init__(self, api_key, reference_id):
        self.api_key = api_key

        self.reference_id = reference_id

        # TODO Call PAS/app.py-> getUserPropsFromAPIKey
        try:
            app_user_instance = AppUserPIM(self.api_key)
            app_data = app_user_instance.get()

            self.store_key = app_data["app_user"]["app_creds"]["store_key"]
            self.a2c_api_key = app_data["app_data"]["app_creds"]["a2c_api_key"]
        except:
            print_exc()
        #Magento
        # self.store_key = "eb7e80c244df8fa6f594b4f07d9cf41a"
        #BigCommerce
        #self.store_key = "b035de9e896fe6ba7762eebdc95a73fe"
        # self.a2c_api_key = "8a71acf63ad0192c8d8f18febd073471"
    def main(self):
        self.pull_channel_products()

        # TODO Product level common config initialisation


    def __del__(self):
        # Call the delete or garbage clear
        print("Do Cleanup")

    def pull_channel_products(self):
        transformer = Transformer('app/product/app_to_pim/denormalisation.json')
        #TODO API calls to Pull products from your channel and merge all the relevant properties to it
        # store the raw object from platform in final_product_list array and update transformations in denormalisation config



        # Configure API key authorization: api_key
        configuration = swagger_client.Configuration()
        configuration.api_key['api_key'] = self.a2c_api_key
        configuration.api_key['store_key'] = self.store_key
        product_api_instance = swagger_client.ProductApi(swagger_client.ApiClient(configuration))
        cart_api_instance = swagger_client.CartApi(swagger_client.ApiClient(configuration))
        category_api_instance = swagger_client.CategoryApi(swagger_client.ApiClient(configuration))
        try:
            products_recieved = []

            #TODO Vary this based on required type of product import
            product_list_dict = {"params": "force_all", "since_id":9303}
            if len(list(product_list_dict.keys())) == 1:
                while True:
                    product_list_api_response = product_api_instance.product_list(**product_list_dict)
                    pprint(product_list_api_response)
                    products_recieved += product_list_api_response.result.product
                    page_cursor = product_list_api_response.pagination.next
                    if page_cursor is not None:
                        product_list_dict["page_cursor"] = page_cursor

                    else:
                        break
            else:
                product_list_api_response = product_api_instance.product_list(**product_list_dict)
                pprint(product_list_api_response)
                products_recieved += product_list_api_response.result.product

            if len(products_recieved) == 0:
                logger.info("No products in the store. Please add products")
                raise Exception


        except Exception as e:
            print_exc()
            logger.error("Product list API failed")
            print("Exception when calling ProductApi->product_list: %s\n" % e)

        #TODO List category
        try:
            start = 0
            count = 250
            params = 'id,parent_id,name,description'
            category_api_response = category_api_instance.category_list(start=start, count=count, params=params)
            category_list = [x.to_dict() for x in category_api_response.result.category]
            build_category_tree_instance = BuildCategoryTree(category_list, self.a2c_api_key, self.store_key)
            category_full_path_map = build_category_tree_instance.build_categories()

        except Exception as e:
            logger.error("Category creation failed", e)
            print_exc()

        final_product_list = []
        products_list = []
        counter = 1


        for product in products_recieved:
            try:
                counter = counter + 1
                # TODO Manage the product level cleanup and final expected custom channel format
                product = product.to_dict()
                product_variants = product.pop("product_options")
                product_id = product['id']
                transformed_product = transformer.transform(product,category_full_path_map)
                #TODO Add option properties in the parent product for identifying the variant properties
                products_list.append(transformed_product)
                if product["type"] == "configurable":

                    product_child_item_response = product_api_instance.product_child_item_list(start=0, count=250,
                                                                                       params="force_all",product_id=product_id)
                    pprint(product_child_item_response)
                    children_products = [x.to_dict() for x in product_child_item_response.result.children]
                    for children_product in children_products:

                        children_product["id"] = children_product["parent_id"] + "_" + children_product["id"]
                        transformed_child_product = transformer.transform(children_product, category_full_path_map, product_variants)

                        transformed_child_product["u_sku"] = transformed_child_product.pop("sku")
                        #TODO review this
                        transformed_child_product["u_model"] = transformed_child_product["u_sku"]
                        products_list.append(transformed_child_product)


            except Exception as e:
                print_exc()
                print(e)

        #TODO Done processing the Denormalisation
        if len(products_list) > 0:
            file_dir = "D:\\Unbxd\\Api2cart_productListFiles\\"

            #TODO GET CART INFO FOR APP NAME

            cart_api_response = cart_api_instance.cart_info()
            pprint(cart_api_response)
            app_name = cart_api_response.result.name.replace(" ","")

            #TODO App name being accessed in wrtiefeedfile
            zip_file_name = write_feed_file(file_dir, products_list, app_name)

            #return "{}/{}/{}".format(self.user.app.name, self.user.identifier, zip_file_name)

            #TODO UPLOAD FILE TO S3 and GET URL
            bucket  = "unbxd-pim-ui"
            file_url = common.utils.file_operations.upload_to_s3(file_dir,zip_file_name, bucket)
            #TODO TRIGGER IMPORT TO PIM API

            response = self.upload_to_pim(file_url, self.api_key)
            return




    def upload_to_pim(self, file_url, api_key):

        url = "http://pimqa-apps.unbxd.io/pim/v1/imports"

        payload = json.dumps({
            "url": f"{file_url}",
            "referenceId": self.reference_id
        })

        headers = {
            'Authorization': api_key,
            'Content-Type': 'application/json',
            'Cookie': 'JSESSIONID=bZRjiDLiKYofPwqieaPNbaJVDUXhIo2pNL74UCvR'
        }

        response = requests.request("POST", url, headers=headers, data=payload)

        return response.text


    def build_categories(self, catList):

        dictfilt = lambda x, y: dict([(i, x[i]) for i in x if i in set(y)])
        categoriesList = []
        cat_keys = ["id", "name", "parent_id"]
        for cat in catList:
            categoriesList.append(dictfilt(cat, cat_keys))

        self.categoryTreePath = self.buildCategoryTree(categoriesList)



### Starting Point for triggering the import

In [ ]:
def trigger_product_import(api_key="", reference_id=""):
    product_importer = ProductImporter(api_key=api_key, reference_id= reference_id)
    product_importer.main()
    return {"data": "Received Import task"}

In [ ]:
trigger_product_import()